# 隐语SecretFlow金融风控全链路能力展示

> This tutorial is only available in Chinese.

> Last updated: Nov 9, 2022
>
> 请使用v0.7.11或以上版本的隐语进行实验。
>
> 以下代码仅作为示例，请勿在生产环境直接使用。

本次实验将会展示如何使用隐语进行在风控领域常用的Logistic Regeression模型和XGB模型的模型研发工作。

隐语接下来将会开放模型部署和在线/离线模型预测功能，敬请期待。

## 实验目标

在本次实验中，我们将会利用一个开源数据集训练一个金融风控场景常用的线性回归和XGB模型。在此过程中将包含以下步骤：

- 样本对齐
- 特征预处理
- 数据分析
- 模型训练
- 模型预测
- 模型评估

请依次执行所有步骤确保实验可以顺利完成。

## 实验前置工作

### 初始化隐语框架

在本次实验中，我们将会包含两个节点：**alice** 和 **bob** . 在真实业务场景，他们将会代表两个不同实体，他们之间的原始数据不被允许直接相互传输，但是他们的原始数据将会被一起用以研发一个模型。

在下面的代码中，我们建立了一个 **SecretFlow Cluster**, 基于 **alice** 和 **bob** 两个节点，我们还创建了三个device：

- alice: PYU device, 负责在alice侧的本地计算，计算输入、计算过程和计算结果仅alice可见
- bob: PYU device, 负责在bob侧的本地计算，计算输入、计算过程和计算结果仅bob可见
- spu: SPU device, 负责alice和bob之间的密态计算，计算输入和计算结果为密态，由alice和bob各掌握一个分片，计算过程为MPC计算，由alice和bob各自的SPU Runtime一起执行。

>  如果你尚未理解以上的一些概念，比如SPU设备，请参考这篇[文档](../developer/design/architecture.md).


In [1]:
import secretflow as sf

sf.shutdown()
sf.init(['alice', 'bob'], address='local')
alice, bob = sf.PYU('alice'), sf.PYU('bob')
spu = sf.SPU(sf.utils.testing.cluster_def(['alice', 'bob']))


/home/fengjun.feng/miniconda3/envs/sf/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


在上面的log中，你应该发现，在**spu**的创建过程中，alice和bob两边都各有一个 **SPURuntime** 被建立并互相创建连接。

### 数据集

本次实验我们采用的原始数据是来自UCI的[Bank Marketing Data Set](https://archive.ics.uci.edu/ml/datasets/bank+marketing). 这个数据集汇集了一家葡萄牙银行机构电话营销的结果。

我们添加了**uid**这一列用于接下来隐私求交的实验。

我们首先看一下数据集所包含的信息。


In [2]:
import pandas as pd

# secretflow.utils.simulation.datasets contains mirrors of some popular open dataset.
from secretflow.utils.simulation.datasets import dataset

df = pd.read_csv(dataset('bank_marketing_full'), sep=';')
df['uid'] = df.index + 1

df


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y,uid
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no,1
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no,2
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no,3
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no,4
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,technician,married,tertiary,no,825,no,no,cellular,17,nov,977,3,-1,0,unknown,yes,45207
45207,71,retired,divorced,primary,no,1729,no,no,cellular,17,nov,456,2,-1,0,unknown,yes,45208
45208,72,retired,married,secondary,no,5715,no,no,cellular,17,nov,1127,5,184,3,success,yes,45209
45209,57,blue-collar,married,secondary,no,668,no,no,telephone,17,nov,508,4,-1,0,unknown,no,45210


该数据集包含了45211个样本，每一个样本代表了一个目标客户。

每个样本包含16个feature，我们这里简单描述一下这个数据集所有的feature。


| feature | 描述 | 取值 |
| :-----| :---- | :---- |
| uid | 客户编码 | 数字 |
| age | 年龄 | 数字 |
| job | 工作类型 |  'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown' |
| marital | 婚姻状况 | 'divorced','married','single','unknown' |
| education | 教育状况 | 'tertiary', 'secondary', 'unknown', 'primary' |
| default | 是否有不良信用记录 | 'no','yes','unknown' |
| housing | 是否有房贷 |  'no','yes','unknown' |
| loan | 是否有个人贷款 | 'no','yes','unknown' |
| contact | 联系方式 | 'cellular','telephone' |
| month | 上次联系月份 | 'jan', 'feb', 'mar', ..., 'nov', 'dec' |
| day | 上次联系月日 |数字|
| duration | 上次沟通时间 | 数字 |
| campaign | 本次活动已经沟通的次数 | 数字 |
| pdays | 距离上次沟通经过的天数 | 数字 |
| previous | 在本次活动之前已经沟通的次数 | 数字 |
| poutcome | 之前活动的结果 | 'unknown', 'failure', 'other', 'success' | 




每个样本的label - y表示对于目标客户的营销结果（是否签订了定额存款合同），取值是'yes','no'。

我们假定以上16个feature由两个机构分别掌握，具体如下。

- alice: age, job, marital, education, default, balance, housing, loan
- bob: contact, day, month, duration, campaign, pdays, previous, poutcome, y


在真实业务场景中, alice和bob所掌握的数据可能是没有对齐的，为了模拟这种情况，我们将数据集shuffle之后，再随机各取90%来模拟这个状况。

In [3]:
import numpy as np

df_alice = df.iloc[:, np.r_[0:8, -1]].sample(frac=0.9)

df_alice


,age,job,marital,education,default,balance,housing,loan,uid
30775,33,technician,single,secondary,no,10,yes,no,30776
38044,55,admin.,divorced,secondary,no,-288,yes,no,38045
10075,60,services,divorced,secondary,no,47,no,no,10076
37076,33,services,married,secondary,no,56,yes,no,37077
11067,52,management,married,tertiary,no,7388,no,yes,11068
...,...,...,...,...,...,...,...,...,...
9083,55,retired,married,primary,no,0,no,no,9084
39576,25,student,single,tertiary,no,241,no,no,39577
25186,40,management,married,tertiary,no,11,yes,no,25187
10436,36,entrepreneur,married,primary,no,6317,no,no,10437


In [4]:
df_bob = df.iloc[:, 8:].sample(frac=0.9)

df_bob


,contact,day,month,duration,campaign,pdays,previous,poutcome,y,uid
11614,unknown,19,jun,211,1,-1,0,unknown,no,11615
24743,cellular,18,nov,150,1,-1,0,unknown,no,24744
42588,cellular,30,dec,158,3,-1,0,unknown,no,42589
4322,unknown,19,may,187,3,-1,0,unknown,no,4323
15930,cellular,22,jul,76,1,-1,0,unknown,no,15931
...,...,...,...,...,...,...,...,...,...,...
30143,cellular,4,feb,76,2,204,3,other,no,30144
16730,cellular,24,jul,149,2,-1,0,unknown,no,16731
35775,cellular,8,may,36,1,-1,0,unknown,no,35776
7050,unknown,28,may,318,2,-1,0,unknown,no,7051


我们这里将df_alice和df_bob保存为文件，作为alice和bob两方的原始输入。

至此，我们完成了所有实验准备工作。

In [5]:
import tempfile

_, alice_path = tempfile.mkstemp()
_, bob_path = tempfile.mkstemp()
df_alice.reset_index(drop=True).to_csv(alice_path, index=False)
df_bob.reset_index(drop=True).to_csv(bob_path, index=False)


(raylet) /home/fengjun.feng/miniconda3/envs/sf/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
(raylet)   warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


## 样本对齐（隐私求交）

显然，第一步我们需要将两边的数据对齐。
隐私求交（[Private Set Intersection](https://en.wikipedia.org/wiki/Private_set_intersection))是一种密码学方法，可以获取两个集合的交集，而不泄露任何其他信息。
在隐语中，SPU设备支持三种隐私求交算法:

- [ECDH](https://ieeexplore.ieee.org/document/6234849/)：半诚实模型, 基于公钥密码学，原本适用于小数据集，但是隐语优化后已经能支持10亿量级的数据。
- [KKRT](https://eprint.iacr.org/2016/799.pdf)：半诚实模型, 基于布谷鸟哈希（Cuckoo Hashing）以及高效不经意传输扩展（OT Extension），适用于大数据集（比如千万数据集）。
- [BC22PCG](https://eprint.iacr.org/2022/334)：半诚实模型, 基于随机相关函数生成器，适用于大数据集。

由于我们这里的数据集较小，我们这里采用的是ECDH方法。


### 方式一：将隐私求交结果保存至文件

在一些应用场景场景中，alice和bob可能在隐私求交之后将结果直接保存至文件中，之后再进行后续操作。这个时候，请调用**psi_csv**接口。

在以下代码中，我们分别制定了两边需要求交的key以及输入和输出路径。

我们需要指定双方的输入文件和输出文件路径。对于ECDH来说，由于双方的地位是平等的，receiver并没有实际含义，你可以任意指定。我们需要设定正确的protocol。sort设为true之后，join的结果将会被排序。

> 请阅读 psi_csv 的文档。

In [6]:
_, alice_psi_path = tempfile.mkstemp()
_, bob_psi_path = tempfile.mkstemp()

spu.psi_csv(
    key="uid",
    input_path={alice: alice_path, bob: bob_path},
    output_path={alice: alice_psi_path, bob: bob_psi_path},
    receiver="alice",
    protocol="ECDH_PSI_2PC",
    sort=True,
)


(raylet) /home/fengjun.feng/miniconda3/envs/sf/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
(raylet)   warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
(SPURuntime pid=45105) I1110 15:04:56.489761 45105 external/com_github_brpc_brpc/src/brpc/server.cpp:1070] Server[yacl::link::internal::ReceiverServiceImpl] is serving on port=23711.
(SPURuntime pid=45105) I1110 15:04:56.489876 45105 external/com_github_brpc_brpc/src/brpc/server.cpp:1073] Check out http://k69b13338.eu95sqa:23711 in web browser.
(SPURuntime pid=45105) I1110 15:04:56.591135 47953 external/com_github_brpc_brpc/src/brpc/socket.cpp:2236] Checking Socket{id=0 addr=127.0.0.1:39345} (0x5604daa51200)
(SPURuntime pid=45106) I1110 15:04:56.641627 45106 external/com_github_brpc_brpc/src/brpc/server.cpp:1070] Server[yacl::link::internal::ReceiverServiceI

(SPURuntime pid=45105) [2022-11-10 15:05:00.520] [info] [bucket_psi.cc:169] bucket size set to 1048576
(SPURuntime pid=45105) [2022-11-10 15:05:00.521] [info] [bucket_psi.cc:77] Begin sanity check for input file: /tmp/tmp0jqbsdge, precheck_switch:true
(SPURuntime pid=45105) [2022-11-10 15:05:00.544] [info] [csv_checker.cc:125] Executing duplicated scripts: LC_ALL=C sort --buffer-size=1G --temporary-directory=/tmp --stable selected-keys.1668063900521197637 | LC_ALL=C uniq -d > duplicate-keys.1668063900521197637
(SPURuntime pid=45105) [2022-11-10 15:05:00.568] [info] [bucket_psi.cc:90] End sanity check for input file: /tmp/tmp0jqbsdge, size=40690
(SPURuntime pid=45105) [2022-11-10 15:05:00.568] [info] [bucket_psi.cc:190] Run psi protocol=1, self_items_count=40690
(SPURuntime pid=45105) [2022-11-10 15:05:00.568] [info] [cryptor_selector.cc:50] Using libSodium
(SPURuntime pid=45105) [2022-11-10 15:05:00.568] [info] [cipher_store.cc:25] Disk cache choose num_bins=64
(SPURuntime pid=45106) [

[{'party': 'alice', 'original_count': 40690, 'intersection_count': 36593},
 {'party': 'bob', 'original_count': 40690, 'intersection_count': 36593}]

### 方式二：将求交结果保存至VDataFrame

VDataFrame是隐语中保存垂直切分数据的数据结构，在接下来的任务中，我们将会不断使用VDataFrame的数据结构。

由于在本次实验中，经过隐私求交之后，我们还有后续操作，所以我们在这里使用 **data.vertical.read_csv** 来将原始数据隐私求交之后的结果直接转化为VDataFrame。

> 请阅读data.vertical.read_csv的文档。很多参数和psi_csv是一致的，这里不再赘述。

In [7]:
from secretflow.data.vertical import read_csv as v_read_csv

vdf = v_read_csv(
    {alice: alice_path, bob: bob_path},
    spu=spu,
    keys="uid",
    drop_keys="uid",
    psi_protocl="ECDH_PSI_2PC",
)
vdf.columns


(SPURuntime pid=45105) [2022-11-10 15:05:02.036] [info] [bucket_psi.cc:119] Begin post filtering, indices.size=36593, should_sort=true
(SPURuntime pid=45105) [2022-11-10 15:05:02.044] [info] [utils.cc:86] Executing sort scripts: tail -n +2 /tmp/tmp-sort-in-1668063902037299176 | LC_ALL=C sort --buffer-size=1G --temporary-directory=./ --stable --field-separator=, --key=9,9 >>/tmp/tmp-sort-out-1668063902037299176
(SPURuntime pid=45105) [2022-11-10 15:05:02.076] [info] [utils.cc:88] Finished sort scripts: tail -n +2 /tmp/tmp-sort-in-1668063902037299176 | LC_ALL=C sort --buffer-size=1G --temporary-directory=./ --stable --field-separator=, --key=9,9 >>/tmp/tmp-sort-out-1668063902037299176, ret=0
(SPURuntime pid=45105) [2022-11-10 15:05:02.076] [info] [bucket_psi.cc:157] End post filtering, in=/tmp/tmp0jqbsdge, out=/tmp/tmprde_yg6t
(SPURuntime pid=45106) [2022-11-10 15:05:02.039] [info] [bucket_psi.cc:119] Begin post filtering, indices.size=36593, should_sort=true
(SPURuntime pid=45106) [2022

Index(['age', 'job', 'marital', 'education', 'default', 'balance', 'housing',
       'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'y'],
      dtype='object')

### 更多

我们在这里展示的是两方单键的隐私求交，隐语也支持三方和多键的隐私求交技术，想要了解更多信息，你可以：

- 阅读这篇[文档](https://www.secretflow.org.cn/docs/spu/en/development/psi.html)了解隐语SPU的隐私求交能力。
- 阅读该[教程](./PSI_On_SPU.ipynb)了解使用的例子。

## 特征预处理

一般情况下，我们都需要对用于建模的数据进行预处理，合理的预处理对模型训练效果非常关键。

在开始特征预处理之前，我们先使用 **stats.table_statistics.table_statistics** 来查看一下特征总体情况，我们会在后面专门讨论全表统计模块。

In [8]:
from secretflow.stats.table_statistics import table_statistics

pd.set_option('display.max_rows', None)
data_stats = table_statistics(vdf)
data_stats

,datatype,total_count,count,count_na,min,max,mean,var,std,sem,...,moment_2,moment_3,moment_4,central_moment_2,central_moment_3,central_moment_4,sum,sum_2,sum_3,sum_4
age,int64,36593,36593,0,18.0,95.0,40.947039,1.128579e+02,10.623461,0.055535,...,1.789515e+03,8.333137e+04,4.121843e+06,1.128548e+02,8.138900e+02,4.203357e+04,1498375.0,6.548372e+07,3.049345e+09,1.508306e+11
job,object,36593,36593,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
marital,object,36593,36593,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
education,object,36593,36593,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
default,object,36593,36593,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
balance,int64,36593,36593,0,-6847.0,102127.0,1362.827563,9.160127e+06,3026.570165,15.821649,...,1.101718e+07,2.688746e+11,4.396339e+15,9.159877e+06,2.288934e+11,1.211695e+16,49869949.0,4.031515e+11,9.838929e+15,-5.145462e+18
housing,object,36593,36593,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
loan,object,36593,36593,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
contact,object,36593,36593,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
day,int64,36593,36593,0,1.0,31.0,15.811795,6.918886e+01,8.317984,0.043483,...,3.191998e+02,7.287211e+03,1.788647e+05,6.918697e+01,5.214820e+01,9.274288e+03,578601.0,1.168048e+07,2.666609e+08,6.545197e+09


In [9]:
pd.reset_option('display.max_rows')

在接下来，我们将会展示隐语以下特征预处理能力：

- 值替换
- 缺失值填充
- WOE分组/分箱转换
- one-hot编码
- 标准化

### 值替换

我们先对以下特征做值替换：

| feature | 描述 | 取值和值替换规则 |
| :-----| :---- | :---- |
| education | 教育状况 | 'tertiary' -> 3, 'secondary' -> 2, 'unknown' -> 0, 'primary' -> 1 |
| default | 是否有不良信用记录 | 'no' -> 0,'yes' -> 1,'unknown' -> NaN |
| housing | 是否有房贷 |  'no' -> 0,'yes' -> 1,'unknown' -> NaN |
| loan | 是否有个人贷款 | 'no' -> 0,'yes' -> 1,'unknown' -> NaN |
| month | 上次联系月份 | 'jan' -> 1, 'feb' -> 2, 'mar' -> 3, ..., 'nov' -> 11, 'dec' ->12 |
| y | label | 'yes' -> 1,'no' -> 0 |


替换完之后，我们使用 **sf.reveal** 来查看效果，请注意在生产中，**sf.reveal** 将会直接泄露数据，需要严格限制和进行审计。

> 在生产中，请严格限制**sf.reveal**的使用。

In [10]:
vdf['education'] = vdf['education'].replace(
    {'tertiary': 3, 'secondary': 2, 'primary': 1, 'unknown': np.NaN}
)

vdf['default'] = vdf['default'].replace({'no': 0, 'yes': 1, 'unknown': np.NaN})

vdf['housing'] = vdf['housing'].replace({'no': 0, 'yes': 1, 'unknown': np.NaN})

vdf['loan'] = vdf['loan'].replace({'no': 0, 'yes': 1, 'unknown': np.NaN})

vdf['month'] = vdf['month'].replace(
    {
        'jan': 1,
        'feb': 2,
        'mar': 3,
        'apr': 4,
        'may': 5,
        'jun': 6,
        'jul': 7,
        'aug': 8,
        'sep': 9,
        'oct': 10,
        'nov': 11,
        'dec': 12,
    }
)

vdf['y'] = vdf['y'].replace(
    {
        'no': 0,
        'yes': 1,
    }
)

print(sf.reveal(vdf.partitions[alice].data))
print(sf.reveal(vdf.partitions[bob].data))


       age            job  marital  education  default  balance  housing  loan
0       43     technician   single        2.0        0      593        1     0
1       46     management  married        3.0        0      229        1     0
2       42     technician  married        2.0        0     8036        0     0
3       38         admin.  married        1.0        0     1487        0     0
4       39    blue-collar  married        2.0        0      138        0     0
...    ...            ...      ...        ...      ...      ...      ...   ...
36588   36  self-employed   single        3.0        0     4844        0     0
36589   49      housemaid  married        1.0        0     3376        0     0
36590   52   entrepreneur  married        3.0        0     1115        1     0
36591   40    blue-collar  married        1.0        0       48        0     0
36592   46       services  married        3.0        0      474        0     0

[36593 rows x 8 columns]
       contact  day  month

#### 安全性讨论

值替换操作由数据所有者的PYU Device执行，不会泄露数据。

### 缺失值填充

接下来我们对缺失值进行填充。我们在这里均填充了众数，其他可选的策略还包括平均数、中位数等。

其他可能的处理方法包括删除缺省的行, 或者可以使用数据完整的行作为训练集，以此来预测缺失值。

替换完之后，我们使用 **sf.reveal** 来查看效果。

In [11]:
vdf["education"] = vdf["education"].fillna(vdf["education"].mode())
vdf["default"] = vdf["default"].fillna(vdf["default"].mode())
vdf["housing"] = vdf["housing"].fillna(vdf["housing"].mode())
vdf["loan"] = vdf["loan"].fillna(vdf["loan"].mode())

print(sf.reveal(vdf.partitions[alice].data))
print(sf.reveal(vdf.partitions[bob].data))


       age            job  marital  education  default  balance  housing  loan
0       43     technician   single        2.0        0      593        1     0
1       46     management  married        3.0        0      229        1     0
2       42     technician  married        2.0        0     8036        0     0
3       38         admin.  married        1.0        0     1487        0     0
4       39    blue-collar  married        2.0        0      138        0     0
...    ...            ...      ...        ...      ...      ...      ...   ...
36588   36  self-employed   single        3.0        0     4844        0     0
36589   49      housemaid  married        1.0        0     3376        0     0
36590   52   entrepreneur  married        3.0        0     1115        1     0
36591   40    blue-collar  married        1.0        0       48        0     0
36592   46       services  married        3.0        0      474        0     0

[36593 rows x 8 columns]
       contact  day  month

#### 安全性讨论

所填充的缺失值由属于数据所有者的PYU Device执行，并在接下来的缺失值操作中由数据所有者的PYU Device使用，不会泄露数据。

### woe分箱

woe分箱用于将连续值替换为离散值。

将连续型特征离散化的一个好处是可以有效地克服数据中隐藏的缺陷： 使模型结果更加稳定。例如，数据中的极端值是影响模型效果的一个重要因素。极端值导致模型参数过高或过低，或导致模型被虚假现象"迷惑"，把原来不存在的关系作为重要模式来学习。而离散化可以有效地减弱极端值和异常值的影响。

变量duration的75%分位数远小于最大值，而且该变量的标准差相对也比较大。因此需要对变量duration进行离散化。

In [12]:
from secretflow.preprocessing.binning.vert_woe_binning import VertWoeBinning
from secretflow.preprocessing.binning.vert_woe_substitution import VertWOESubstitution

binning = VertWoeBinning(spu)
woe_rules = binning.binning(
    vdf,
    binning_method="chimerge",
    bin_num=4,
    bin_names={alice: [], bob: ["duration"]},
    label_name="y",
)

woe_sub = VertWOESubstitution()
vdf = woe_sub.substitution(vdf, woe_rules)

print(sf.reveal(vdf.partitions[alice].data))
print(sf.reveal(vdf.partitions[bob].data))


(raylet) /home/fengjun.feng/miniconda3/envs/sf/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
(raylet)   warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
(raylet) /home/fengjun.feng/miniconda3/envs/sf/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
(raylet)   warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
(raylet) /home/fengjun.feng/miniconda3/envs/sf/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
(raylet)   warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported 

       age            job  marital  education  default  balance  housing  loan
0       43     technician   single        2.0        0      593        1     0
1       46     management  married        3.0        0      229        1     0
2       42     technician  married        2.0        0     8036        0     0
3       38         admin.  married        1.0        0     1487        0     0
4       39    blue-collar  married        2.0        0      138        0     0
...    ...            ...      ...        ...      ...      ...      ...   ...
36588   36  self-employed   single        3.0        0     4844        0     0
36589   49      housemaid  married        1.0        0     3376        0     0
36590   52   entrepreneur  married        3.0        0     1115        1     0
36591   40    blue-collar  married        1.0        0       48        0     0
36592   46       services  married        3.0        0      474        0     0

[36593 rows x 8 columns]
       contact  day  month

(_run pid=56064) WARNING:absl:No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


#### 安全性讨论

woe分桶需要利用alice和bob两边的数据，因此相关的计算需要使用**SPU device**确保原始数据不被泄露。

### One Hot编码

one-hot编码适用于将类型编码转化为数值编码。 对于job、marital等特征我们需要one-hot编码。

In [13]:
from secretflow.preprocessing.encoder import OneHotEncoder

encoder = OneHotEncoder()
# for vif and correlation only
vdf_hat = vdf.drop(columns=["job", "marital", "contact", "month", "day", "poutcome"])

tranformed_df = encoder.fit_transform(vdf['job'])
vdf[tranformed_df.dtypes.index] = tranformed_df

tranformed_df = encoder.fit_transform(vdf['marital'])
vdf[tranformed_df.dtypes.index] = tranformed_df

tranformed_df = encoder.fit_transform(vdf['contact'])
vdf[tranformed_df.dtypes.index] = tranformed_df

tranformed_df = encoder.fit_transform(vdf['month'])
vdf[tranformed_df.dtypes.index] = tranformed_df

tranformed_df = encoder.fit_transform(vdf['day'])
vdf[tranformed_df.dtypes.index] = tranformed_df

tranformed_df = encoder.fit_transform(vdf['poutcome'])
vdf[tranformed_df.dtypes.index] = tranformed_df

vdf = vdf.drop(columns=["job", "marital", "contact", "month", "day", "poutcome"])

print(sf.reveal(vdf.partitions[alice].data))
print(sf.reveal(vdf.partitions[bob].data))


(raylet) /home/fengjun.feng/miniconda3/envs/sf/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
(raylet)   warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
(raylet) /home/fengjun.feng/miniconda3/envs/sf/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
(raylet)   warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


       age  education  default  balance  housing  loan  job_admin.  \
0       43        2.0        0      593        1     0         0.0   
1       46        3.0        0      229        1     0         0.0   
2       42        2.0        0     8036        0     0         0.0   
3       38        1.0        0     1487        0     0         1.0   
4       39        2.0        0      138        0     0         0.0   
...    ...        ...      ...      ...      ...   ...         ...   
36588   36        3.0        0     4844        0     0         0.0   
36589   49        1.0        0     3376        0     0         0.0   
36590   52        3.0        0     1115        1     0         0.0   
36591   40        1.0        0       48        0     0         0.0   
36592   46        3.0        0      474        0     0         0.0   

       job_blue-collar  job_entrepreneur  job_housemaid  ...  job_retired  \
0                  0.0               0.0            0.0  ...          0.0   
1    

#### 安全性讨论

one-hot编码操作由数据所有者的PYU Device执行，不会泄露数据。

### 标准化 
特征之间数值差距太大会使得模型收敛困难，我们一般先对数值进行标准化。

In [14]:
from secretflow.preprocessing import StandardScaler

X = vdf.drop(columns=['y'])
y = vdf['y']
scaler = StandardScaler()
X = scaler.fit_transform(X)
vdf[X.columns] = X
print(sf.reveal(vdf.partitions[alice].data))
print(sf.reveal(vdf.partitions[bob].data))


(_run pid=57413) /home/fengjun.feng/miniconda3/envs/sf/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but StandardScaler was fitted without feature names
(_run pid=57413)   warnings.warn(
(_run pid=57412) /home/fengjun.feng/miniconda3/envs/sf/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but StandardScaler was fitted without feature names
(_run pid=57412)   warnings.warn(


            age  education   default   balance   housing      loan  \
0      0.193250  -0.214015 -0.134056 -0.254360  0.890792 -0.437727   
1      0.475648   1.317062 -0.134056 -0.374630  0.890792 -0.437727   
2      0.099118  -0.214015 -0.134056  2.204893 -1.122596 -0.437727   
3     -0.277412  -1.745093 -0.134056  0.041028 -1.122596 -0.437727   
4     -0.183280  -0.214015 -0.134056 -0.404697 -1.122596 -0.437727   
...         ...        ...       ...       ...       ...       ...   
36588 -0.465677   1.317062 -0.134056  1.150219 -1.122596 -0.437727   
36589  0.758046  -1.745093 -0.134056  0.665175 -1.122596 -0.437727   
36590  1.040444   1.317062 -0.134056 -0.081885  0.890792 -0.437727   
36591 -0.089147  -1.745093 -0.134056 -0.434434 -1.122596 -0.437727   
36592  0.475648   1.317062 -0.134056 -0.293679 -1.122596 -0.437727   

       job_admin.  job_blue-collar  job_entrepreneur  job_housemaid  ...  \
0       -0.361290        -0.524802         -0.185399      -0.168477  ...   
1      

#### 安全性讨论

标准化操作由数据所有者的PYU Device执行，不会泄露数据。

### 更多

隐语还支持其他更多的特征预处理能力，请参考这篇[文档](./data_preprocessing_with_data_frame.ipynb).

至此，我们已经完成了所有特征预处理工作。

> 本文主要目的是为了展示隐语的预处理能力，本文对于数据预处理方法的使用可能是有争议的，敬请谅解。

## 数据分析

在建模之前，我们有必要分析一下我们所使用的数据，以便确认是否需要重复特征预处理的过程。

下面我们将会展示隐语以下数据分析能力:

- 全表统计
- 相关系数矩阵
- VIF指标计算


### 全表统计

我们提供了类似于 **pd.DataFrame.describe** 来展示所有特征的基本统计信息。

> 在特征预处理的过程中，你可以不断调用全表统计来关注预处理效果。

In [15]:
from secretflow.stats.table_statistics import table_statistics

pd.set_option('display.max_rows', None)
data_stats = table_statistics(vdf)
data_stats


,datatype,total_count,count,count_na,min,max,mean,var,std,sem,...,moment_2,moment_3,moment_4,central_moment_2,central_moment_3,central_moment_4,sum,sum_2,sum_3,sum_4
duration,float64,36593,36593,0,-1.990727,1.902071,-6.601933e-18,1.000027,1.000014,0.005228,...,1.000000,-0.557851,2.899545,1.000000,-0.557851,2.899545,-2.415845e-13,36593.0,-2.041344e+04,1.061031e+05
campaign,float64,36593,36593,0,-0.566486,19.310148,-7.611640e-17,1.000027,1.000014,0.005228,...,1.000000,4.947549,43.236502,1.000000,4.947549,43.236502,-2.785328e-12,36593.0,1.810457e+05,1.582153e+06
pdays,float64,36593,36593,0,-0.412100,8.074647,3.728150e-17,1.000027,1.000014,0.005228,...,1.000000,2.618954,9.979817,1.000000,2.618954,9.979817,1.364242e-12,36593.0,9.583538e+04,3.651915e+05
previous,float64,36593,36593,0,-0.243363,114.104096,4.349509e-17,1.000027,1.000014,0.005228,...,1.000000,44.783657,4683.146470,1.000000,44.783657,4683.146470,1.591616e-12,36593.0,1.638768e+06,1.713704e+08
y,int64,36593,36593,0,0.000000,1.000000,1.162517e-01,0.102740,0.320531,0.001676,...,0.116252,0.116252,0.116252,0.102737,0.078851,0.071072,4.254000e+03,4254.0,4.254000e+03,4.254000e+03
contact_cellular,float64,36593,36593,0,-1.357755,0.736510,-7.456301e-17,1.000027,1.000014,0.005228,...,1.000000,-0.621246,1.385946,1.000000,-0.621246,1.385946,-2.728484e-12,36593.0,-2.273325e+04,5.071593e+04
contact_telephone,float64,36593,36593,0,-0.260775,3.834729,4.349509e-17,1.000027,1.000014,0.005228,...,1.000000,3.573955,13.773154,1.000000,3.573955,13.773154,1.591616e-12,36593.0,1.307817e+05,5.040010e+05
contact_unknown,float64,36593,36593,0,-0.636008,1.572308,0.000000e+00,1.000027,1.000014,0.005228,...,1.000000,0.936300,1.876657,1.000000,0.936300,1.876657,0.000000e+00,36593.0,3.426201e+04,6.867251e+04
month_1.0,float64,36593,36593,0,-0.179805,5.561570,9.320376e-18,1.000027,1.000014,0.005228,...,1.000000,5.381765,29.963395,1.000000,5.381765,29.963395,3.410605e-13,36593.0,1.969349e+05,1.096450e+06
month_2.0,float64,36593,36593,0,-0.249288,4.011427,6.524263e-17,1.000027,1.000014,0.005228,...,1.000000,3.762139,15.153690,1.000000,3.762139,15.153690,2.387424e-12,36593.0,1.376680e+05,5.545190e+05


In [16]:
pd.reset_option('display.max_rows')


#### 安全性讨论

请注意，全表统计会暴露数据整体统计结果，其背后实际上蕴含了**sf.reveal**,请谨慎使用。

### 相关系数矩阵

我们接下来计算特征和特征之间，特征和标签之间的相关系数矩阵。

> 计算相关系数矩阵时，one-hot编码各列无需参与计算。

In [17]:
from secretflow.stats.ss_pearsonr_v import PearsonR

pearson_r_calculator = PearsonR(spu)
corr_matrix = pearson_r_calculator.pearsonr(vdf_hat)

import numpy as np

np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
corr_matrix


(_run pid=56064) /home/fengjun.feng/miniconda3/envs/sf/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but StandardScaler was fitted without feature names
(_run pid=56064)   warnings.warn(
(_run pid=57413) /home/fengjun.feng/miniconda3/envs/sf/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but StandardScaler was fitted without feature names
(_run pid=57413)   warnings.warn(


(_run pid=56064) [2022-11-10 15:05:24.416] [info] [thread_pool.cc:30] Create a fixed thread pool with size 63
(_run pid=57413) [2022-11-10 15:05:24.835] [info] [thread_pool.cc:30] Create a fixed thread pool with size 63


(_run pid=57413) WARNING:absl:No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)
(_spu_compile pid=44652) WARNING:absl:No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


array([[1.000, -0.185, 0.013, 0.010, 0.331, -0.007, -0.001, -0.005,
        0.016, 0.003, -0.008],
       [-0.185, 1.000, -0.091, -0.032, -0.074, 0.004, 0.002, 0.013,
        -0.012, -0.023, 0.013],
       [0.013, -0.091, 1.000, 0.439, 0.104, -0.021, 0.005, -0.031, 0.003,
        0.123, -0.022],
       [0.010, -0.032, 0.439, 1.000, 0.091, 0.003, 0.022, -0.019, 0.015,
        0.037, -0.009],
       [0.331, -0.074, 0.104, 0.091, 1.000, 0.022, 0.069, -0.023, 0.053,
        -0.135, -0.068],
       [-0.007, 0.004, -0.021, 0.003, 0.022, 1.000, -0.169, -0.016,
        0.096, -0.183, -0.014],
       [-0.001, 0.002, 0.005, 0.022, 0.069, -0.169, 1.000, -0.012, 0.068,
        -0.070, -0.030],
       [-0.005, 0.013, -0.031, -0.019, -0.023, -0.016, -0.012, 1.000,
        -0.067, -0.010, 0.079],
       [0.016, -0.012, 0.003, 0.015, 0.053, 0.096, 0.068, -0.067, 1.000,
        -0.072, -0.084],
       [0.003, -0.023, 0.123, 0.037, -0.135, -0.183, -0.070, -0.010,
        -0.072, 1.000, 0.043],
       [-

#### 安全性讨论

相关系数矩阵的计算需要利用alice和bob两边的数据，因此相关的计算需要使用**SPU device**确保原始数据不被泄露。

### VIF指标计算

隐语还支持VIF的计算来进行多重共线性检验。

> 计算VIF指标时，one-hot编码各列无需参与计算。

In [18]:
from secretflow.stats.ss_vif_v import VIF

vif_calculator = VIF(spu)
vif_results = vif_calculator.vif(vdf_hat)
print(vdf_hat.columns)
print(vif_results)


(_run pid=44652) /home/fengjun.feng/miniconda3/envs/sf/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but StandardScaler was fitted without feature names
(_run pid=44652)   warnings.warn(
(_run pid=57413) /home/fengjun.feng/miniconda3/envs/sf/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but StandardScaler was fitted without feature names
(_run pid=57413)   warnings.warn(


(_run pid=44652) [2022-11-10 15:05:25.323] [info] [thread_pool.cc:30] Create a fixed thread pool with size 63
Index(['duration', 'campaign', 'pdays', 'previous', 'y', 'age', 'education',
       'default', 'balance', 'housing', 'loan'],
      dtype='object')
[1.162 1.044 1.276 1.243 1.177 1.082 1.052 1.011 1.030 1.089 1.018]


#### 安全性讨论

VIF指标的计算需要利用alice和bob两边的数据，因此相关的计算需要使用**SPU device**确保原始数据不被泄露。

## 模型训练

接下来，我们将会分别训练一个逻辑回归模型和一个XGB模型。


### 随机分割

在训练之前，我们需要将数据分割为训练集和验证集。

其中train_x和train_y为训练集的特征和标签。test_x和test_y为训练集的特征和标签。


In [19]:
from secretflow.data.split import train_test_split

random_state = 1234

train_vdf, test_vdf = train_test_split(vdf, train_size=0.8, random_state=random_state)

train_x = train_vdf.drop(columns=['y'])
train_y = train_vdf['y']

test_x = test_vdf.drop(columns=['y'])
test_y = test_vdf['y']


#### 安全性讨论

随机分割时，每一方会共享随机数种子，并由每一方数据的owner分别执行各自的数据分割并且确保最终分割结果仍然是对齐的。

### PSI（人群稳定性分析）

样本稳定指数是衡量样本变化所产生的偏移量的一种重要指标，通常用来衡量样本的稳定程度，比如样本在两个月份之间的变化是否稳定。通常变量的PSI值在0.1以下表示变化不太显著，在0.1到0.25之间表示有比较显著的变化，大于0.25表示变量变化比较剧烈，需要特殊关注。

接下来以`balance`为例子，确认两次抽样的样本分布是否接近。

> 根据业务需求，PSI分析也可以在数据分析或者特征预处理的时候进行。


In [20]:
stats_df = table_statistics(train_x['balance'])

In [21]:
min_val, max_val = stats_df['min'], stats_df['max']

In [22]:
from secretflow.stats import psi_eval
from secretflow.stats.core.utils import equal_range
import jax.numpy as jnp

split_points = equal_range(jnp.array([min_val, max_val]), 3)
balance_psi_score = psi_eval(train_x['balance'], test_x['balance'], split_points)

sf.reveal(balance_psi_score)

DeviceArray(0.000, dtype=float32)

#### 安全性讨论

PSI分析是一个单方运算，由数据owner的PYU Device执行计算。

### 逻辑回归模型

使用 **ml.linear.ss_sgd.SSRegression** 可以进行密态逻辑回归模型的训练。

请参考相关的API文档。


In [23]:
from secretflow.ml.linear.ss_sgd import SSRegression

lr_model = SSRegression(spu)
lr_model.fit(
    x=train_x,
    y=train_y,
    epochs=3,
    learning_rate=0.1,
    batch_size=1024,
    sig_type='t1',
    reg_type='logistic',
    penalty='l2',
    l2_norm=0.5,
)


(_spu_compile pid=44652) /* error: missing value */
(_spu_compile pid=44652) {}:task_name:_spu_compile


(_run pid=57412) WARNING:absl:No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)
(raylet) /home/fengjun.feng/miniconda3/envs/sf/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
(raylet)   warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
(raylet) /home/fengjun.feng/miniconda3/envs/sf/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
(raylet)   warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


你可能会对为何上面的语句很快就执行完毕感到困惑，原因是在隐语中，语句都是lazy evaluation的，在上面的例子中，直到lr_model被真正被使用的时候，**lr_model.fit**才会被执行。

#### 安全性讨论

SSRegression的训练基于SPU Device，双方的原始数据将会被保护。

### XGBoost模型

使用 **ml.boost.ss_xgb_v.Xgb** 可以进行密态XGBoost模型的训练。

请参考相关的API文档。

In [24]:
from secretflow.ml.boost.ss_xgb_v import Xgb

xgb = Xgb(spu)
params = {
    'num_boost_round': 3,
    'max_depth': 5,
    'sketch_eps': 0.25,
    'objective': 'logistic',
    'reg_lambda': 0.2,
    'subsample': 1,
    'colsample_bytree': 1,
    'base_score': 0.5,
}
xgb_model = xgb.train(params=params, dtrain=train_x, label=train_y)


(_spu_compile pid=57412) /* error: missing value */
(_spu_compile pid=57412) {}


(_run pid=68105) WARNING:absl:No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)
(raylet) /home/fengjun.feng/miniconda3/envs/sf/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
(raylet)   warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
(raylet) /home/fengjun.feng/miniconda3/envs/sf/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
(raylet)   warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


(_run pid=57412) [2022-11-10 15:05:38.196] [info] [thread_pool.cc:30] Create a fixed thread pool with size 63


(_run pid=68077) WARNING:absl:No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


(_run pid=68077) [2022-11-10 15:05:40.288] [info] [thread_pool.cc:30] Create a fixed thread pool with size 63
(_spu_compile pid=68105) /* error: missing value */
(_spu_compile pid=68105) {}:task_name:_run


...

(_spu_compile pid=75168) WARNING:absl:No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)
(_run pid=72626) WARNING:absl:No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)
(_run pid=72627) WARNING:absl:No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)
(_run pid=72628) 2022-11-10 15:05:59.676639: E external/org_tensorflow/tensorflow/core/tpu/tpu_initializer_helper.cc:230] Unable to open shared memory for GCS file system creator.
(_run pid=72628) WARNING:absl:No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)
(_run pid=75170) WARNING:absl:No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)
(_run pid=72636) WARNING:absl:No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)
(_run pid=75169) WARNING:absl:No GPU/TPU found, falling back to CPU. (Set 

(_run pid=72628) [2022-11-10 15:06:10.337] [info] [thread_pool.cc:30] Create a fixed thread pool with size 63
(_run pid=75169) [2022-11-10 15:06:10.540] [info] [thread_pool.cc:30] Create a fixed thread pool with size 63
(_spu_compile pid=72628) /* error: missing value */
(_spu_compile pid=72628) {}:task_name:_run


(raylet) /home/fengjun.feng/miniconda3/envs/sf/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
(raylet)   warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
(raylet) /home/fengjun.feng/miniconda3/envs/sf/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
(raylet)   warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
(raylet) /home/fengjun.feng/miniconda3/envs/sf/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
(raylet)   warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported 

(_run pid=89768) [2022-11-10 15:06:41.189] [info] [thread_pool.cc:30] Create a fixed thread pool with size 63
(_run pid=89766) [2022-11-10 15:06:41.335] [info] [thread_pool.cc:30] Create a fixed thread pool with size 63
(_spu_compile pid=89768) /* error: missing value */
(_spu_compile pid=89768) {}:task_name:_run


(raylet) /home/fengjun.feng/miniconda3/envs/sf/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
(raylet)   warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
(raylet) /home/fengjun.feng/miniconda3/envs/sf/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
(raylet)   warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
(raylet) /home/fengjun.feng/miniconda3/envs/sf/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
(raylet)   warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported 

Xgb.train将会直接执行，请耐心等待。

#### 安全性讨论

Xgb的训练基于SPU Device，双方的原始数据将会被保护。

## 模型预测

接下来，我们将会分别利用刚刚训练好的模型来预测测试集。

### 逻辑回归模型

由于在我们的场景下，数据集标签的持有者是bob，因此我们在这里将预测结果**reveal**给bob.

In [25]:
lr_y_hat = lr_model.predict(x=test_x, batch_size=1024, to_pyu=bob)

#### 安全性讨论

逻辑回归的预测基于SPU Device，双方的原始数据将会被保护。

当设置**to_pyu**，预测结果将会被reveal给该方，否则将仍然保持秘密分享的状态。

### XGBoost模型

由于在我们的场景下，数据集标签的持有者是bob，因此我们在这里将预测结果**reveal**给bob.

In [26]:
xgb_y_hat = xgb_model.predict(dtrain=test_x, to_pyu=bob)


(_run pid=101755) [2022-11-10 15:07:12.900] [info] [thread_pool.cc:30] Create a fixed thread pool with size 63
(_run pid=102626) [2022-11-10 15:07:12.900] [info] [thread_pool.cc:30] Create a fixed thread pool with size 63


(raylet) /home/fengjun.feng/miniconda3/envs/sf/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
(raylet)   warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
(raylet) /home/fengjun.feng/miniconda3/envs/sf/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
(raylet)   warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


#### 安全性讨论

XGBoost模型的预测基于SPU Device，双方的原始数据将会被保护。

当设置**to_pyu**，预测结果将会被reveal给该方，否则将仍然保持秘密分享的状态。

## 模型评估

接下来，我们将利用测试数据集对模型效果进行评估，包括：

- 二分类评估
- PVA
- P-Value
- 评分卡转换

### 二分类评估

隐语中对二分类的评估有集成的支持。

`BiClassificationEval` 将计算 `AUC`, `KS`, `F1 Score`, `Lift`, `K-S`, `Gain`, `Precision`, `Recall` 等统计数值， 并提供（基于prediction score的）等频和等距分箱的统计报告和总报告。

不同分桶中评估模型的预测的`threshold`不同。总报告中依赖`threshold`的统计取的是各个分桶的最佳值。

详情可以参考API文档。

In [27]:
from secretflow.stats.biclassification_eval import BiClassificationEval

biclassification_evaluator = BiClassificationEval(
    y_true=test_y, y_score=lr_y_hat, bucket_size=20
)
lr_report = sf.reveal(biclassification_evaluator.get_all_reports())


(raylet) /home/fengjun.feng/miniconda3/envs/sf/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
(raylet)   warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
(raylet) /home/fengjun.feng/miniconda3/envs/sf/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
(raylet)   warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
(raylet) /home/fengjun.feng/miniconda3/envs/sf/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
(raylet)   warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported 

(_run pid=117896) [2022-11-10 15:07:21.122] [info] [thread_pool.cc:30] Create a fixed thread pool with size 63


(_run pid=117896) WARNING:absl:No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)
(_spu_compile pid=101746) WARNING:absl:No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)
(_spu_compile pid=115860) WARNING:absl:No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)
(_spu_compile pid=115859) WARNING:absl:No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


(_run pid=101746) [2022-11-10 15:07:21.342] [info] [thread_pool.cc:30] Create a fixed thread pool with size 63


In [28]:
print(f'positive_samples: {lr_report.summary_report.positive_samples}')
print(f'negative_samples: {lr_report.summary_report.negative_samples}')
print(f'total_samples: {lr_report.summary_report.total_samples}')
print(f'auc: {lr_report.summary_report.auc}')
print(f'ks: {lr_report.summary_report.ks}')
print(f'f1_score: {lr_report.summary_report.f1_score}')


positive_samples: 860.0
negative_samples: 6459.0
total_samples: 7319.0
auc: 0.8958946466445923
ks: 0.6431037187576294
f1_score: 0.5415411591529846


In [29]:
biclassification_evaluator = BiClassificationEval(
    y_true=test_y, y_score=xgb_y_hat, bucket_size=20
)
xgb_report = sf.reveal(biclassification_evaluator.get_all_reports())


In [30]:
print(f'positive_samples: {xgb_report.summary_report.positive_samples}')
print(f'negative_samples: {xgb_report.summary_report.negative_samples}')
print(f'total_samples: {xgb_report.summary_report.total_samples}')
print(f'auc: {xgb_report.summary_report.auc}')
print(f'ks: {xgb_report.summary_report.ks}')
print(f'f1_score: {xgb_report.summary_report.f1_score}')


positive_samples: 860.0
negative_samples: 6459.0
total_samples: 7319.0
auc: 0.8133864402770996
ks: 0.5009485483169556
f1_score: 0.4135618805885315


### PVA (预测和实际平均值比较)

结果由`abs(mean(Acutal) - mean(Prediction))`计算获得, 值越小越好。

In [31]:
from secretflow.stats import pva_eval

lr_pva_score = pva_eval(test_y, lr_y_hat, 1)

sf.reveal(lr_pva_score)


DeviceArray(0.051, dtype=float32)

In [32]:
xgb_pva_score = pva_eval(test_y, xgb_y_hat, 1)

sf.reveal(xgb_pva_score)


DeviceArray(0.065, dtype=float32)

### P-Value
双方可通过p-value的值来判断参数是否显著，即该自变量是否可以有效预测因变量的变异, 从而判定对应的解释变量是否应包括在模型中。


In [33]:
from secretflow.stats import SSPValue

model = lr_model.save_model()
sspv = SSPValue(spu)
pvalues = sspv.pvalues(test_x, test_y, model)

pvalues

(_run pid=117458) [2022-11-10 15:08:18.378] [info] [thread_pool.cc:30] Create a fixed thread pool with size 63


(raylet) /home/fengjun.feng/miniconda3/envs/sf/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
(raylet)   warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
(raylet) /home/fengjun.feng/miniconda3/envs/sf/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
(raylet)   warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


(_run pid=9587) [2022-11-10 15:08:21.419] [info] [thread_pool.cc:30] Create a fixed thread pool with size 63


(_run pid=9587) WARNING:absl:No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)
(_spu_compile pid=117457) WARNING:absl:No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)
(_run pid=9615) WARNING:absl:No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


(_run pid=9615) [2022-11-10 15:08:21.780] [info] [thread_pool.cc:30] Create a fixed thread pool with size 63


array([0.000, 0.682, 0.855, 0.755, 0.981, 0.989, 0.975, 0.964, 0.986,
       0.773, 0.963, 0.981, 0.977, 0.973, 0.975, 0.859, 0.833, 0.982,
       0.827, 0.946, 0.993, 0.994, 0.994, 0.987, 0.984, 0.979, 0.990,
       0.986, 0.947, 0.997, 0.978, 0.970, 0.997, 0.976, 0.994, 0.961,
       0.994, 0.967, 0.979, 0.997, 0.986, 0.975, 0.997, 0.987, 0.987,
       0.969, 0.997, 0.975, 0.962, 0.990, 0.976, 0.992, 0.819, 0.977,
       0.645, 0.416, 0.805, 0.452, 0.024, 0.201, 0.998, 0.992, 0.986,
       0.974, 0.999, 0.965, 0.998, 0.988, 0.919, 0.998, 0.988, 0.987,
       0.998, 0.990, 0.987, 0.000])

### 评分卡转换

> 严格来说，评分卡转化是对预测结果的后续处理，并不属于模型评估。


我们将 `y = 1` 的概率设为`p`， `odds = p / (1 - p)`, 评分卡设定的分值刻度可以通过将分值表示为比率对数的线性表达式来定义，即可表示为下式：

`Score = A - B log(odds)`， A 和 B 是可以设定的常数。隐语中提供了评分卡转换功能，详情可以参考API文档。

In [34]:
from secretflow.stats import BiClassificationEval, ScoreCard

sc = ScoreCard(20, 600, 20)
score = sc.transform(xgb_y_hat)

sf.reveal(score.partitions[bob])


array([[496.647],
       [453.126],
       [496.647],
       ...,
       [451.232],
       [494.179],
       [466.241]])

### 安全性讨论

以上所有模型评估的方法均为单方运算，由label拥有者的PYU Device进行运算。

## 实验结束

最后，我们需要清理临时文件，并关闭隐语cluster。

In [35]:
import os

try:
    os.remove(alice_path)
    os.remove(alice_psi_path)
    os.remove(bob_path)
    os.remove(bob_psi_path)
except OSError:
    pass

sf.shutdown()

恭喜！你已经完成了隐语金融风控全链路的全部实验内容。

如果你对本实验有任何建议和问题，请在[Github Issues](https://github.com/secretflow/secretflow/issues)上联系我们。